In [1]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler

stringInputs = ["Actor1Name", "Actor2Name", "EventCode", "Actor1Geo_FullName", "Actor2Geo_FullName", "SOURCEURL"]

indexer = [StringIndexer(inputCol=column, outputCol=column + "_IX") for column in stringInputs]

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features', labelCol='label', maxIter=10)